In [ ]:
from pyneqsys.symbolic import SymbolicSys

In [ ]:
def f(x):
    return [(x[0] - x[1])**3/2 + x[0] - 1,
            (x[1] - x[0])**3/2 + x[1]]
neqsys = SymbolicSys.from_callback(f, 2)

In [ ]:
neqsys.solve([1, 0])

In [ ]:
neqsys.solve([1, 0], solver='kinsol')

In [ ]:
import scipy
version = scipy.__version__.split('.')
major = int(version[0])
minor = int(version[1])
if major > 0 or minor >= 17:
    # New solver in SciPy v0.17
    # http://scipy.github.io/devdocs/generated/scipy.optimize.root.html#scipy.optimize.root
    class SciPyLsq:
        def __init__(self, neqsys):
            self.neqsys = neqsys
            
        def __call__(self, x0, **kwargs):
            new_kwargs = kwargs.copy()
            if self.neqsys.band is not None:
                raise ValueError("Not supported (see SciPy docs)")
            new_kwargs['args'] = (self.neqsys.internal_params,)
            return scipy.optimize.least_squares(self.neqsys.f_callback, x0, jac=self.neqsys.j_callback, **new_kwargs)
    result = neqsys.solve([1, 0], attached_solver=SciPyLsq)
    print(result)

In [ ]:
def plot_convergence(attached_solver, plot_attr):
    import numpy as np
    import matplotlib.pyplot as plt
    %matplotlib inline
    x_history = np.array(attached_solver.history_x)
    plt.figure(figsize=(15, 3))
    plt.subplot(1, 4, 1)
    plt.plot(x_history[:, 0], x_history[:, 1]); plt.xlabel('x0'), plt.ylabel('x1')
    plt.subplot(1, 4, 2)
    plt.plot(neqsys.rms(x_history)); plt.xlabel('iteration'), plt.ylabel('RMS(residuals)')
    plt.subplot(1, 4, 3)
    plt.semilogy(range(15, len(x_history)), neqsys.rms(x_history[15:])); plt.xlabel('iteration'), plt.ylabel('RMS(residuals)')
    plt.subplot(1, 4, 4)
    plt.plot(np.asarray(getattr(attached_solver, plot_attr)))
    plt.ylabel(plot_attr)
    plt.xlabel('iteration')
    plt.tight_layout()

In [ ]:
from pyneqsys.solvers import LineSearchingGradientDescentSolver as LSGD
lsgd = LSGD()
print(neqsys.solve([.8, .1], attached_solver=lsgd))
plot_convergence(lsgd, 'history_rms_f')

In [ ]:
from pyneqsys.solvers import PolakRibiereConjugateGradientSolver as CG
cg = CG(4)
print(neqsys.solve([.8, .1], attached_solver=cg))
plot_convergence(cg, 'history_sn')

In [ ]:
from pyneqsys.solvers import AutoDampedGradientDescentSolver as ADGD
adgd = ADGD(.15)
print(neqsys.solve([.8, .1], attached_solver=adgd))
plot_convergence(adgd, 'history_damping')